In [1]:
import pandas as pd
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import scipy
import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import norm, uniform, skew, kurtosis
from sklearn.metrics import r2_score
import statsmodels.api as sm
import os
from arch.univariate import ConstantMean, GARCH, Normal
from arch.__future__ import reindexing
from arch.univariate import arch_model
from arch import arch_model
from arch.univariate import GARCH
from arch.__future__ import reindexing
import yfinance as yf
import wrds
import tensorflow

In [2]:
data = pd.read_csv(r'C:\FinancialEcon Machine Learning\data.csv')

In [3]:
data['DATE'] = pd.to_datetime(data['DATE'], format='%Y%m%d')
data = data.sort_values(['permno', 'DATE'])
data['shifted_monthly_return'] = data.groupby('permno')['monthly_return'].shift(-1)
data.dropna(subset=['shifted_monthly_return'], inplace=True)

In [4]:
data

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,date,monthly_return,shifted_monthly_return
1182646,10000,1986-02-28,1.610000e+04,NaN,NaN,NaN,NaN,NaN,0.211159,NaN,...,1.244051e-06,0.250000,0.065278,1.231289,2.120805,4.785175e-08,39.0,19860228,-0.257143,0.365385
1188901,10000,1986-03-31,1.196000e+04,NaN,NaN,NaN,NaN,NaN,0.262471,-0.257143,...,1.891760e-06,0.044776,0.031004,1.021089,1.079774,1.023392e-07,39.0,19860331,0.365385,-0.098592
1195185,10000,1986-04-30,1.633000e+04,NaN,NaN,NaN,7.897668,NaN,0.326987,0.365385,...,7.315091e-07,0.145161,0.044548,1.033817,1.745333,7.467463e-08,39.0,19860430,-0.098592,-0.222656
1201471,10000,1986-05-30,1.517200e+04,NaN,NaN,NaN,8.472954,NaN,0.347950,-0.098592,...,1.215981e-06,0.022727,0.011246,1.184555,1.502285,7.649551e-08,39.0,19860530,-0.222656,-0.005025
1207775,10000,1986-06-30,1.179386e+04,NaN,NaN,NaN,8.250098,NaN,0.357205,-0.222656,...,2.744328e-06,0.115702,0.038863,0.959128,1.756198,7.360224e-08,39.0,19860630,-0.005025,-0.080808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4083224,93436,2021-07-30,6.688269e+08,1.467523,2.153624,-2.304367,19.135683,0.087168,0.916212,0.087137,...,1.179508e-12,0.052685,0.024500,0.266461,6.999865,3.696248e-09,37.0,20210730,0.011034,0.070605
4089930,93436,2021-08-31,6.803383e+08,1.461826,2.136934,-1.907103,19.223606,0.085674,0.848168,0.011034,...,1.108349e-12,0.046941,0.020875,0.265651,5.539199,4.593460e-09,37.0,20210831,0.070605,0.054042
4096679,93436,2021-09-30,7.370200e+08,1.454301,2.114992,-0.266406,18.968600,0.084132,0.647541,0.070605,...,1.302815e-12,0.038279,0.021313,0.275420,5.274786,5.209458e-09,37.0,20210930,0.054042,0.436530
4103484,93436,2021-10-29,7.785819e+08,1.436531,2.063623,-0.395890,19.031238,0.079950,0.652689,0.054042,...,7.818319e-13,0.027533,0.015922,0.273759,4.970871,5.366691e-09,37.0,20211029,0.436530,0.027612


# Question 2: 

3 factor IPCA for monthly returns. Report R^2, relative pricing error, and spread ptf sharpe ratio. 



In [9]:
pip install ipca

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.6 MB 4.6 MB/s eta 0:00:01
   ----------- ---------------------------- 0.8/2.6 MB 8.1 MB/s eta 0:00:01
   --------------------- ------------------ 1.4/2.6 MB 9.8 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   - -------------------------------------- 0.7/28.1 MB 14.8 MB/s eta 0:00:02
   -- ------------------------------------- 1.4/28.1 MB 15.1 MB/s eta 0:00:02
   --- ------------------------------------ 2.2/28.1 MB 15.7 MB/s eta 0:00:02
   ---- ----------------------------------- 3.0/28.1 MB 15.7 MB/s eta 0:00:02
   ----- ---------------------------------- 3.8/28.1 MB 16.4 MB/s

In [33]:
import ipca 


# Set MultiIndex
data.set_index(['permno', 'DATE'], inplace=True)
characteristics = data.drop(columns=['monthly_return', 'shifted_monthly_return']).copy()
returns = data['monthly_return'].copy()


ipca_model = ipca.InstrumentedPCA(n_factors=3, max_iter=10000, iter_tol=1e-05, n_jobs=-1)#init ipca

ipca_model.fit(characteristics, returns, data_type='portfolio')
#extract factors and loagins
gamma, factors = ipca_model.get_factors()

The panel dimensions are:
n_samples: 3352 , L: 96 , T: 443


[========================================================================] 100%


Step 1 - Aggregate Update: 6201382.869629591
Step 2 - Aggregate Update: 2579.8006345759236
Step 3 - Aggregate Update: 1228.8495261156918
Step 4 - Aggregate Update: 667.3417089884499
Step 5 - Aggregate Update: 782.7607730353245
Step 6 - Aggregate Update: 956.8021768986655
Step 7 - Aggregate Update: 1156.4072425395534
Step 8 - Aggregate Update: 9963.891551697114
Step 9 - Aggregate Update: 1473.4199396202475
Step 10 - Aggregate Update: 1574.4406980953881
Step 11 - Aggregate Update: 1720.8696463482065
Step 12 - Aggregate Update: 1991.6636611249596
Step 13 - Aggregate Update: 2410.8637822977926
Step 14 - Aggregate Update: 2956.3247367719814
Step 15 - Aggregate Update: 3585.2458812427576
Step 16 - Aggregate Update: 4262.773453198133
Step 17 - Aggregate Update: 4969.998181110317
Step 18 - Aggregate Update: 5704.046717045043
Step 19 - Aggregate Update: 6475.164112476552
Step 20 - Aggregate Update: 7300.359453742771
Step 21 - Aggregate Update: 8195.041048436586
Step 22 - Aggregate Update: 9178.

In [37]:
total_R2 = ipca_model.score(characteristics, returns, indices=indices, data_type='portfolio')

The panel dimensions are:
n_samples: 3352 , L: 96 , T: 443


$$R^2$$: 0.9008365483801241


#

# Question 3: Split stocks into Market Capitilazation groups. 

In [6]:
data['']

Index(['permno', 'DATE', 'mvel1', 'beta', 'betasq', 'chmom', 'dolvol',
       'idiovol', 'indmom', 'mom1m', 'mom6m', 'mom12m', 'mom36m', 'pricedelay',
       'turn', 'absacc', 'acc', 'age', 'agr', 'bm', 'bm_ia', 'cashdebt',
       'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv',
       'chpmia', 'convind', 'currat', 'depr', 'divi', 'divo', 'dy', 'egr',
       'ep', 'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'invest', 'lev',
       'lgr', 'mve_ia', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat',
       'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt',
       'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'ps',
       'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'roic', 'salecash',
       'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp',
       'tang', 'tb', 'aeavol', 'cash', 'chtx', 'cinvest', 'ear', 'nincr',
       'roaq', 'roavol', 'roeq', 'rsup', 'stdacc', 'stdcf', 'ms', 'baspread',
       'ill', 'maxret', 'retvol', '

In [5]:
#median mkt cap
median_mvel1 = data['mvel1'].median()

stock_above_median = data[data['mvel1'] > median_mvel1]
stock_below_media = data[data['mvel1'] < median_mvel1]

In [9]:
data.set_index(['permno', 'DATE'], inplace=True)

mvel1      beta    betasq     chmom     dolvol  \
permno DATE                                                                
10000  1986-02-28  1.610000e+04       NaN       NaN       NaN        NaN   
       1986-03-31  1.196000e+04       NaN       NaN       NaN        NaN   
       1986-04-30  1.633000e+04       NaN       NaN       NaN   7.897668   
       1986-05-30  1.517200e+04       NaN       NaN       NaN   8.472954   
       1986-06-30  1.179386e+04       NaN       NaN       NaN   8.250098   
...                         ...       ...       ...       ...        ...   
93436  2021-07-30  6.688269e+08  1.467523  2.153624 -2.304367  19.135683   
       2021-08-31  6.803383e+08  1.461826  2.136934 -1.907103  19.223606   
       2021-09-30  7.370200e+08  1.454301  2.114992 -0.266406  18.968600   
       2021-10-29  7.785819e+08  1.436531  2.063623 -0.395890  19.031238   
       2021-11-30  1.118751e+09  1.508117  2.274417 -0.258012  19.042739   

                    idiovol    indmom     mom1m     mom6m    mom12m  ...  \
permno DATE                                                          ...   
10000  1986-02-28       NaN  0.211159       NaN       NaN       NaN  ...   
       1986-03-31       NaN  0.262471 -0.257143       NaN       NaN  ...   
       1986-04-30       NaN  0.326987  0.365385       NaN       NaN  ...   
       1986-05-30       NaN  0.347950 -0.098592       NaN       NaN  ...   
       1986-06-30       NaN  0.357205 -0.222656       NaN       NaN  ...   
...                     ...       ...       ...       ...       ...  ...   
93436  2021-07-30  0.087168  0.916212  0.087137 -0.114005  1.895046  ...   
       2021-08-31  0.085674  0.848168  0.011034 -0.143448  1.375311  ...   
       2021-09-30  0.084132  0.647541  0.070605  0.017321  0.379034  ...   
       2021-10-29  0.079950  0.652689  0.054042  0.101493  0.714925  ...   
       2021-11-30  0.079768  0.594775  0.436530  0.093087  0.998454  ...   

                            ill    maxret    retvol  std_dolvol   std_turn  \
permno DATE                                                                  
10000  1986-02-28  1.244051e-06  0.250000  0.065278    1.231289   2.120805   
       1986-03-31  1.891760e-06  0.044776  0.031004    1.021089   1.079774   
       1986-04-30  7.315091e-07  0.145161  0.044548    1.033817   1.745333   
       1986-05-30  1.215981e-06  0.022727  0.011246    1.184555   1.502285   
       1986-06-30  2.744328e-06  0.115702  0.038863    0.959128   1.756198   
...                         ...       ...       ...         ...        ...   
93436  2021-07-30  1.179508e-12  0.052685  0.024500    0.266461   6.999865   
       2021-08-31  1.108349e-12  0.046941  0.020875    0.265651   5.539199   
       2021-09-30  1.302815e-12  0.038279  0.021313    0.275420   5.274786   
       2021-10-29  7.818319e-13  0.027533  0.015922    0.273759   4.970871   
       2021-11-30  7.860388e-13  0.126616  0.028839    0.551016  14.071951   

                      zerotrade  sic2      date  monthly_return  \
permno DATE                                                       
10000  1986-02-28  4.785175e-08  39.0  19860228       -0.257143   
       1986-03-31  1.023392e-07  39.0  19860331        0.365385   
       1986-04-30  7.467463e-08  39.0  19860430       -0.098592   
       1986-05-30  7.649551e-08  39.0  19860530       -0.222656   
       1986-06-30  7.360224e-08  39.0  19860630       -0.005025   
...                         ...   ...       ...             ...   
93436  2021-07-30  3.696248e-09  37.0  20210730        0.011034   
       2021-08-31  4.593460e-09  37.0  20210831        0.070605   
       2021-09-30  5.209458e-09  37.0  20210930        0.054042   
       2021-10-29  5.366691e-09  37.0  20211029        0.436530   
       2021-11-30  3.974110e-09  37.0  20211130        0.027612   

                   shifted_monthly_return  
permno DATE                                
10000  1986-02-28                0.365385  
       1986-03-31            

In [15]:
stock_below_media.set_index(['permno', 'DATE'], inplace=True)
stock_below_media

mvel1      beta    betasq     chmom     dolvol  \
permno DATE                                                                
10000  1986-02-28  16100.000000       NaN       NaN       NaN        NaN   
       1986-03-31  11960.000000       NaN       NaN       NaN        NaN   
       1986-04-30  16330.000000       NaN       NaN       NaN   7.897668   
       1986-05-30  15172.000000       NaN       NaN       NaN   8.472954   
       1986-06-30  11793.859375       NaN       NaN       NaN   8.250098   
...                         ...       ...       ...       ...        ...   
93435  2011-11-30  28398.161366  1.384676  1.917327  0.008131  10.560200   
       2011-12-30  26966.319886  1.515213  2.295870  0.084353  10.793303   
       2012-01-31  23864.000000  1.312074  1.721537  0.651790  11.256850   
       2012-02-29  30307.279545  1.111136  1.234622  0.318042  10.871460   
       2012-03-30  35796.000000  1.192657  1.422431  0.737430  11.287014   

                    idiovol    indmom     mom1m     mom6m    mom12m  ...  \
permno DATE                                                          ...   
10000  1986-02-28       NaN  0.211159       NaN       NaN       NaN  ...   
       1986-03-31       NaN  0.262471 -0.257143       NaN       NaN  ...   
       1986-04-30       NaN  0.326987  0.365385       NaN       NaN  ...   
       1986-05-30       NaN  0.347950 -0.098592       NaN       NaN  ...   
       1986-06-30       NaN  0.357205 -0.222656       NaN       NaN  ...   
...                     ...       ...       ...       ...       ...  ...   
93435  2011-11-30  0.115161 -0.004038  0.367816 -0.697917 -0.826241  ...   
       2011-12-30  0.117563  0.134582 -0.050420 -0.524000 -0.799660  ...   
       2012-01-31  0.130545  0.001914 -0.115044 -0.058333 -0.829047  ...   
       2012-02-29  0.129846 -0.046581  0.270000 -0.512195 -0.852000  ...   
       2012-03-30  0.127995 -0.035852  0.181102 -0.180645 -0.809578  ...   

                            ill    maxret    retvol  std_dolvol   std_turn  \
permno DATE                                                                  
10000  1986-02-28  1.244051e-06  0.250000  0.065278    1.231289   2.120805   
       1986-03-31  1.891760e-06  0.044776  0.031004    1.021089   1.079774   
       1986-04-30  7.315091e-07  0.145161  0.044548    1.033817   1.745333   
       1986-05-30  1.215981e-06  0.022727  0.011246    1.184555   1.502285   
       1986-06-30  2.744328e-06  0.115702  0.038863    0.959128   1.756198   
...                         ...       ...       ...         ...        ...   
93435  2011-11-30  2.544599e-07  0.255319  0.078058    0.637865   5.725523   
       2011-12-30  2.590575e-07  0.335135  0.105107    1.113941  26.299255   
       2012-01-31  1.462451e-07  0.272727  0.072207    0.750968  11.798107   
       2012-02-29  1.082887e-07  0.066553  0.035900    0.628297   8.763378   
       2012-03-30  5.891809e-08  0.096296  0.044259    0.852552  17.209074   

                      zerotrade  sic2      date  monthly_return  \
permno DATE                                                       
10000  1986-02-28  4.785175e-08  39.0  19860228       -0.257143   
       1986-03-31  1.023392e-07  39.0  19860331        0.365385   
       1986-04-30  7.467463e-08  39.0  19860430       -0.098592   
       1986-05-30  7.649551e-08  39.0  19860530       -0.222656   
       1986-06-30  7.360224e-08  39.0  19860630       -0.005025   
...                         ...   ...       ...             ...   
93435  2011-11-30  1.215001e-08  29.0  20111130       -0.050420   
       2011-12-30  7.257591e-09  29.0  20111230       -0.115044   
       2012-01-31  9.442442e-09  29.0  20120131        0.270000   
       2012-02-29  8.310076e-09  29.0  20120229        0.181102   
       2012-03-30  5.837502e-09  29.0  20120330        0.793333   

                   shifted_monthly_return  
permno DATE                                
10000  1986-02-28                0.365385  
       1986-03-31            

In [16]:
import ipca 

def estimate_ipca(data):
    characteristics = data.drop(columns=['monthly_return', 'shifted_monthly_return', 'mvel1']).copy()
    returns = data['monthly_return'].copy()
    ipca_model = ipca.InstrumentedPCA(n_factors=3, max_iter=10000, iter_tol=1e-05, n_jobs=-1)
    ipca_model.fit(characteristics, returns, data_type='portfolio')
    gamma, factors = ipca_model.get_factors()
    total_R2 = ipca_model.score(characteristics, returns, data_type='portfolio')
    return total_R2

total_R2_above = estimate_ipca(stock_above_median)

total_R2_below = estimate_ipca(stock_below_media)

print(f"Total R^2 for stocks above median market cap (mvel1): {total_R2_above}")
print(f"Total R^2 for stocks below median market cap (mvel1): {total_R2_below}")

The panel dimensions are:
n_samples: 2322 , L: 95 , T: 443


[========================================================================] 100%


Step 1 - Aggregate Update: 6063030.554453602
Step 2 - Aggregate Update: 23480.31352887713
Step 3 - Aggregate Update: 22370.54703052171
Step 4 - Aggregate Update: 16950.194841577373
Step 5 - Aggregate Update: 19800.934189910393
Step 6 - Aggregate Update: 14263.4725754442
Step 7 - Aggregate Update: 3387.7322102583857
Step 8 - Aggregate Update: 7071.264809847005
Step 9 - Aggregate Update: 15051.832199979091
Step 10 - Aggregate Update: 21957.32383192039
Step 11 - Aggregate Update: 30082.575102872866
Step 12 - Aggregate Update: 40699.34897445081
Step 13 - Aggregate Update: 51525.19314118993
Step 14 - Aggregate Update: 58780.132060142874
Step 15 - Aggregate Update: 60870.18484626972
Step 16 - Aggregate Update: 56438.47213840965
Step 17 - Aggregate Update: 47409.12071853096
Step 18 - Aggregate Update: 37547.173226563435
Step 19 - Aggregate Update: 28962.406680483313
Step 20 - Aggregate Update: 22116.943493989587
Step 21 - Aggregate Update: 16829.73039282352
Step 22 - Aggregate Update: 12790.1

The panel dimensions are:
n_samples: 2322 , L: 95 , T: 443


[========================================================================] 100%


...

[========================================================================] 100%


Step 1 - Aggregate Update: 8366500.197610148
Step 2 - Aggregate Update: 16289.917469532043
Step 3 - Aggregate Update: 3277.5854879875174
Step 4 - Aggregate Update: 8925.017407289915
Step 5 - Aggregate Update: 5867.796612023618
Step 6 - Aggregate Update: 7336.29541330155
Step 7 - Aggregate Update: 10253.627487080794
Step 8 - Aggregate Update: 10668.991723154075
Step 9 - Aggregate Update: 8767.882165908166
Step 10 - Aggregate Update: 6564.7551420954405
Step 11 - Aggregate Update: 5032.104597161546
Step 12 - Aggregate Update: 4185.839381826212
Step 13 - Aggregate Update: 3769.0765050908667
Step 14 - Aggregate Update: 3539.928257065054
Step 15 - Aggregate Update: 3348.820970761866
Step 16 - Aggregate Update: 3129.777471900612
Step 17 - Aggregate Update: 2867.0383432192757
Step 18 - Aggregate Update: 2566.4473073902773
Step 19 - Aggregate Update: 2239.1525748160348
Step 20 - Aggregate Update: 1895.1918502462795
Step 21 - Aggregate Update: 1542.448839027129
Step 22 - Aggregate Update: 1187.4

The panel dimensions are:
n_samples: 1991 , L: 95 , T: 443


[========================================================================] 100%


Total R^2 for stocks above median market cap (mvel1): 0.5794763114303744
Total R^2 for stocks below median market cap (mvel1): -110.10590865086539


it appears that the stocks with smaller market cap are much harder to predict that those above median marekt cap. This might be because there is less data in the parameters for the smaller stocks, and they're much more volatile, thus more unpredictable. 